# Librairies Import

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import _pickle as cPickle

# Pre-processing Function

### To replace NaN values

In [59]:
#  mediane si quantative
#  mod si qualitative
cabin = df['Cabin'].unique()[5]
def remplaceNanValues(df, columnName) :
    if columnName == 'Age' :
        x = df[columnName].median()
    elif columnName == 'Cabin' :
        x = cabin
    else :
        x = df[columnName].unique()[0]
        
    for index, row in df.iterrows() :
        if pd.isna(row[columnName]):
            df.loc[index, columnName] = x        

In [43]:
def transformToString(df,columnName) :
    df[columnName] = df[columnName].apply(str)

In [44]:
def chooseAndDummifyVariables(df, features_columns, to_dummify, targetName):
    X = df[features_columns]
    y = df[targetName]    
    X = pd.get_dummies(X, columns = to_dummify)
    
    return X,y

# Model train, score and save Function

In [45]:
from sklearn.model_selection import train_test_split

#### To save the Model

In [46]:
def saveModel(model) :
    filename = ''+model+'.sav'
    cPickle.dump(model, open(filename, 'wb'))

#### Train and score the Model

In [229]:
def trainAndScore(model, X, y):
    liste_test_size = [a/20.0 for a in range(0,20)][1:]
    scores = []
    for ts in liste_test_size :
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = ts, random_state=0)
        clf = model.fit(X_train, y_train)
        #clf = model.fit(X_train, y_train, eval_set=(X_test,y_test))
        score = clf.score(X_test, y_test)
        scores.append(score)
    print("[INFO] SCORES : ",scores)
    print("[INFO] MEAN = ",np.array(scores).mean())
    print("[INFO] MAX = ",max(scores))

# Importing Datas

In [143]:
df = pd.read_csv('data/train.csv', sep=',')

In [144]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [145]:
df.count()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            714
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          204
Embarked       889
dtype: int64

### Specific Function

In [146]:
def getTitle(str):
    return str.split(',')[1].split('.')[0].split(' ')[1]

# Features selection

In [147]:
df['Pclass'].unique()

array([3, 1, 2], dtype=int64)

In [148]:
df['Parch'].unique()

array([0, 1, 2, 5, 3, 4, 6], dtype=int64)

In [149]:
df['Sex'].unique()

array(['male', 'female'], dtype=object)

In [150]:
df['SibSp'].unique()

array([1, 0, 3, 4, 2, 5, 8], dtype=int64)

In [151]:
features_columns = ['Pclass', 'Parch', 'Sex', 'Age', 'SibSp', 'Fare']
to_dummify = ['Pclass', 'Parch', 'Sex', 'SibSp']
targetName = 'Survived'

In [153]:
remplaceNanValues(df, 'Age')
remplaceNanValues(df, 'Cabin')
remplaceNanValues(df, 'Embarked')

In [154]:
df.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [155]:
X, y = chooseAndDummifyVariables(df, features_columns, to_dummify, targetName)

In [156]:
X.head()

,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,...,Parch_6,Sex_female,Sex_male,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8
0,22.0,7.2500,0,0,1,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
1,38.0,71.2833,1,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
2,26.0,7.9250,0,0,1,1,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
3,35.0,53.1000,1,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
4,35.0,8.0500,0,0,1,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0


# Features Engineering

In [157]:
X['is_child'] = X['Age'] <= 11

In [158]:
X['Title'] = df['Name'].apply(lambda x : getTitle(x) )

In [159]:
X['Surname'] = df['Name'].apply(lambda x : '(' in x)

In [161]:
X = pd.get_dummies(X, columns=['Title'])

In [162]:
X.head()

,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,...,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the
0,22.0,7.2500,0,0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,38.0,71.2833,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,26.0,7.9250,0,0,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,35.0,53.1000,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,35.0,8.0500,0,0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


# Model build

## Logistic Regression

In [242]:
from sklearn.linear_model import LogisticRegression

In [243]:
model = LogisticRegression()

#### To ignore FutureWarning

In [244]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [245]:
trainAndScore(model, X, y)

[INFO] SCORES :  [0.8444444444444444, 0.8333333333333334, 0.8059701492537313, 0.7988826815642458, 0.8116591928251121, 0.8059701492537313, 0.8012820512820513, 0.7955182072829131, 0.7955112219451371, 0.8026905829596412, 0.8085539714867617, 0.8037383177570093, 0.8120689655172414, 0.7996794871794872, 0.7937219730941704, 0.788218793828892, 0.7981530343007915, 0.7967581047381546, 0.7638724911452184]
[INFO] MEAN =  0.803159323852214
[INFO] MAX =  0.8444444444444444


## SVM

In [88]:
from sklearn.svm import SVC

In [170]:
model = SVC(gamma='auto')

In [171]:
trainAndScore(model, X, y)

[INFO] SCORES :  [0.7777777777777778, 0.7333333333333333, 0.7611940298507462, 0.7653631284916201, 0.7668161434977578, 0.7649253731343284, 0.7467948717948718, 0.7366946778711485, 0.7281795511221946, 0.7242152466367713, 0.7331975560081466, 0.7214953271028037, 0.7120689655172414, 0.7099358974358975, 0.6920777279521674, 0.6605890603085554, 0.6701846965699209, 0.6633416458852868, 0.5879574970484062]
[INFO] MEAN =  0.718744342491525
[INFO] MAX =  0.7777777777777778


# Naive Bayes

In [172]:
from sklearn.naive_bayes import GaussianNB

In [173]:
model = GaussianNB()

In [174]:
trainAndScore(model, X, y)

[INFO] SCORES :  [0.8666666666666667, 0.7777777777777778, 0.8059701492537313, 0.8100558659217877, 0.8161434977578476, 0.8171641791044776, 0.8173076923076923, 0.7871148459383753, 0.7955112219451371, 0.7556053811659192, 0.7128309572301426, 0.394392523364486, 0.3931034482758621, 0.3958333333333333, 0.414050822122571, 0.6774193548387096, 0.7572559366754618, 0.7805486284289277, 0.7296340023612751]
[INFO] MEAN =  0.7002308570773779
[INFO] MAX =  0.8666666666666667


# K-Nearest Neighbors

In [226]:
from sklearn.neighbors import KNeighborsClassifier

In [237]:
model = KNeighborsClassifier(n_neighbors=25) 

In [238]:
trainAndScore(model, X, y)

[INFO] SCORES :  [0.8, 0.7333333333333333, 0.7388059701492538, 0.7541899441340782, 0.757847533632287, 0.7425373134328358, 0.7115384615384616, 0.7030812324929971, 0.7206982543640897, 0.7107623318385651, 0.7128309572301426, 0.7046728971962617, 0.6913793103448276, 0.6618589743589743, 0.6831091180866966, 0.6802244039270687, 0.6609498680738787, 0.6658354114713217, 0.47461629279811096]
[INFO] MEAN =  0.7004353478106938
[INFO] MAX =  0.8


#  Gradient Boosting Algorithms

In [199]:
from sklearn.ensemble import GradientBoostingClassifier

In [200]:
model = GradientBoostingClassifier(n_estimators=100,max_depth=5)

In [201]:
trainAndScore(model, X, y)

[INFO] SCORES :  [0.8444444444444444, 0.8333333333333334, 0.8283582089552238, 0.8379888268156425, 0.8295964125560538, 0.8097014925373134, 0.8237179487179487, 0.8291316526610645, 0.8154613466334164, 0.8094170403587444, 0.8065173116089613, 0.8037383177570093, 0.7879310344827586, 0.782051282051282, 0.7757847533632287, 0.7363253856942497, 0.762532981530343, 0.7231920199501247, 0.6953955135773318]
[INFO] MEAN =  0.7965589108962354
[INFO] MAX =  0.8444444444444444


# Using Grid Search CV to find the best Params

In [211]:
from sklearn.model_selection import GridSearchCV

rfc = GradientBoostingClassifier(n_estimators=20) 

param_grid = { 
    'n_estimators': [40, 200],
    'max_depth' : [2, 40],
}

CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 10)
CV_rfc.fit(X, y)
print(CV_rfc.best_params_)

{'max_depth': 2, 'n_estimators': 200}


In [198]:
model = GradientBoostingClassifier(n_estimators=200,max_depth=2)
trainAndScore(model, X, y)

[INFO] SCORES :  [0.8444444444444444, 0.8222222222222222, 0.7835820895522388, 0.8100558659217877, 0.8161434977578476, 0.8283582089552238, 0.8044871794871795, 0.803921568627451, 0.8054862842892768, 0.804932735426009, 0.8085539714867617, 0.8074766355140187, 0.8103448275862069, 0.7852564102564102, 0.796711509715994, 0.7755960729312763, 0.7928759894459103, 0.7169576059850374, 0.7154663518299882]
[INFO] MEAN =  0.7964670248123834
[INFO] MAX =  0.8444444444444444


#  XGBoost

In [208]:
from xgboost import XGBClassifier

In [209]:
model = XGBClassifier()

In [210]:
trainAndScore(model, X, y)

[INFO] SCORES :  [0.8666666666666667, 0.8222222222222222, 0.8059701492537313, 0.8268156424581006, 0.8340807174887892, 0.8283582089552238, 0.8237179487179487, 0.8207282913165266, 0.8179551122194514, 0.8094170403587444, 0.8167006109979633, 0.8074766355140187, 0.7948275862068965, 0.8028846153846154, 0.7982062780269058, 0.788218793828892, 0.7810026385224275, 0.7718204488778054, 0.7733175914994097]
[INFO] MEAN =  0.810020378869281
[INFO] MAX =  0.8666666666666667


# CatBoostClassifier

In [212]:
from catboost import Pool, CatBoostClassifier, cv

In [222]:
model = CatBoostClassifier(eval_metric='Accuracy',use_best_model=True,random_seed=0)

In [223]:
trainAndScore(model, X, y)

Learning rate set to 0.07667
0:	learn: 0.8120567	test: 0.7777778	best: 0.7777778 (0)	total: 42.9ms	remaining: 42.9s
1:	learn: 0.8250591	test: 0.8222222	best: 0.8222222 (1)	total: 86.5ms	remaining: 43.1s
2:	learn: 0.8203310	test: 0.8222222	best: 0.8222222 (1)	total: 114ms	remaining: 37.9s
3:	learn: 0.8191489	test: 0.8000000	best: 0.8222222 (1)	total: 147ms	remaining: 36.5s
4:	learn: 0.8215130	test: 0.8222222	best: 0.8222222 (1)	total: 174ms	remaining: 34.7s
5:	learn: 0.8226950	test: 0.8222222	best: 0.8222222 (1)	total: 202ms	remaining: 33.5s
6:	learn: 0.8215130	test: 0.8222222	best: 0.8222222 (1)	total: 233ms	remaining: 33.1s
7:	learn: 0.8215130	test: 0.8000000	best: 0.8222222 (1)	total: 259ms	remaining: 32.2s
8:	learn: 0.8191489	test: 0.8000000	best: 0.8222222 (1)	total: 289ms	remaining: 31.8s
9:	learn: 0.8215130	test: 0.8000000	best: 0.8222222 (1)	total: 317ms	remaining: 31.4s
10:	learn: 0.8226950	test: 0.8000000	best: 0.8222222 (1)	total: 346ms	remaining: 31.1s
11:	learn: 0.8238771	t

99:	learn: 0.8581560	test: 0.8444444	best: 0.8444444 (13)	total: 2.91s	remaining: 26.2s
100:	learn: 0.8593381	test: 0.8444444	best: 0.8444444 (13)	total: 2.94s	remaining: 26.2s
101:	learn: 0.8593381	test: 0.8444444	best: 0.8444444 (13)	total: 2.97s	remaining: 26.2s
102:	learn: 0.8593381	test: 0.8444444	best: 0.8444444 (13)	total: 3s	remaining: 26.1s
103:	learn: 0.8581560	test: 0.8444444	best: 0.8444444 (13)	total: 3.03s	remaining: 26.1s
104:	learn: 0.8581560	test: 0.8666667	best: 0.8666667 (104)	total: 3.06s	remaining: 26.1s
105:	learn: 0.8581560	test: 0.8666667	best: 0.8666667 (104)	total: 3.1s	remaining: 26.1s
106:	learn: 0.8593381	test: 0.8666667	best: 0.8666667 (104)	total: 3.12s	remaining: 26.1s
107:	learn: 0.8593381	test: 0.8666667	best: 0.8666667 (104)	total: 3.15s	remaining: 26s
108:	learn: 0.8593381	test: 0.8666667	best: 0.8666667 (104)	total: 3.18s	remaining: 26s
109:	learn: 0.8593381	test: 0.8666667	best: 0.8666667 (104)	total: 3.19s	remaining: 25.8s
110:	learn: 0.8593381	te

193:	learn: 0.8877069	test: 0.8444444	best: 0.8666667 (104)	total: 5.84s	remaining: 24.3s
194:	learn: 0.8877069	test: 0.8444444	best: 0.8666667 (104)	total: 5.87s	remaining: 24.2s
195:	learn: 0.8877069	test: 0.8444444	best: 0.8666667 (104)	total: 5.9s	remaining: 24.2s
196:	learn: 0.8877069	test: 0.8444444	best: 0.8666667 (104)	total: 5.93s	remaining: 24.2s
197:	learn: 0.8877069	test: 0.8444444	best: 0.8666667 (104)	total: 5.96s	remaining: 24.2s
198:	learn: 0.8877069	test: 0.8444444	best: 0.8666667 (104)	total: 6s	remaining: 24.1s
199:	learn: 0.8877069	test: 0.8444444	best: 0.8666667 (104)	total: 6.03s	remaining: 24.1s
200:	learn: 0.8841608	test: 0.8444444	best: 0.8666667 (104)	total: 6.07s	remaining: 24.1s
201:	learn: 0.8853428	test: 0.8444444	best: 0.8666667 (104)	total: 6.1s	remaining: 24.1s
202:	learn: 0.8853428	test: 0.8444444	best: 0.8666667 (104)	total: 6.13s	remaining: 24.1s
203:	learn: 0.8877069	test: 0.8444444	best: 0.8666667 (104)	total: 6.16s	remaining: 24s
204:	learn: 0.887

286:	learn: 0.8959811	test: 0.8444444	best: 0.8666667 (104)	total: 8.76s	remaining: 21.8s
287:	learn: 0.8959811	test: 0.8444444	best: 0.8666667 (104)	total: 8.79s	remaining: 21.7s
288:	learn: 0.8959811	test: 0.8444444	best: 0.8666667 (104)	total: 8.82s	remaining: 21.7s
289:	learn: 0.8959811	test: 0.8444444	best: 0.8666667 (104)	total: 8.85s	remaining: 21.7s
290:	learn: 0.8959811	test: 0.8444444	best: 0.8666667 (104)	total: 8.89s	remaining: 21.7s
291:	learn: 0.8959811	test: 0.8444444	best: 0.8666667 (104)	total: 8.92s	remaining: 21.6s
292:	learn: 0.8959811	test: 0.8444444	best: 0.8666667 (104)	total: 8.96s	remaining: 21.6s
293:	learn: 0.8959811	test: 0.8444444	best: 0.8666667 (104)	total: 8.99s	remaining: 21.6s
294:	learn: 0.8959811	test: 0.8444444	best: 0.8666667 (104)	total: 9.02s	remaining: 21.5s
295:	learn: 0.8959811	test: 0.8444444	best: 0.8666667 (104)	total: 9.04s	remaining: 21.5s
296:	learn: 0.8959811	test: 0.8444444	best: 0.8666667 (104)	total: 9.07s	remaining: 21.5s
297:	learn

378:	learn: 0.9030733	test: 0.8444444	best: 0.8666667 (104)	total: 11.5s	remaining: 18.8s
379:	learn: 0.9030733	test: 0.8444444	best: 0.8666667 (104)	total: 11.5s	remaining: 18.8s
380:	learn: 0.9030733	test: 0.8444444	best: 0.8666667 (104)	total: 11.5s	remaining: 18.7s
381:	learn: 0.9030733	test: 0.8444444	best: 0.8666667 (104)	total: 11.6s	remaining: 18.7s
382:	learn: 0.9030733	test: 0.8444444	best: 0.8666667 (104)	total: 11.6s	remaining: 18.7s
383:	learn: 0.9030733	test: 0.8444444	best: 0.8666667 (104)	total: 11.6s	remaining: 18.7s
384:	learn: 0.9042553	test: 0.8444444	best: 0.8666667 (104)	total: 11.7s	remaining: 18.7s
385:	learn: 0.9042553	test: 0.8444444	best: 0.8666667 (104)	total: 11.7s	remaining: 18.6s
386:	learn: 0.9042553	test: 0.8444444	best: 0.8666667 (104)	total: 11.8s	remaining: 18.6s
387:	learn: 0.9042553	test: 0.8444444	best: 0.8666667 (104)	total: 11.8s	remaining: 18.6s
388:	learn: 0.9042553	test: 0.8444444	best: 0.8666667 (104)	total: 11.8s	remaining: 18.6s
389:	learn

...

584:	learn: 1.0000000	test: 0.7473436	best: 0.7933884 (64)	total: 13.3s	remaining: 9.44s
585:	learn: 1.0000000	test: 0.7473436	best: 0.7933884 (64)	total: 13.3s	remaining: 9.42s
586:	learn: 1.0000000	test: 0.7473436	best: 0.7933884 (64)	total: 13.4s	remaining: 9.4s
587:	learn: 1.0000000	test: 0.7473436	best: 0.7933884 (64)	total: 13.4s	remaining: 9.38s
588:	learn: 1.0000000	test: 0.7473436	best: 0.7933884 (64)	total: 13.4s	remaining: 9.35s
589:	learn: 1.0000000	test: 0.7473436	best: 0.7933884 (64)	total: 13.4s	remaining: 9.33s
590:	learn: 1.0000000	test: 0.7473436	best: 0.7933884 (64)	total: 13.4s	remaining: 9.3s
591:	learn: 1.0000000	test: 0.7461629	best: 0.7933884 (64)	total: 13.5s	remaining: 9.28s
592:	learn: 1.0000000	test: 0.7473436	best: 0.7933884 (64)	total: 13.5s	remaining: 9.26s
593:	learn: 1.0000000	test: 0.7473436	best: 0.7933884 (64)	total: 13.5s	remaining: 9.24s
594:	learn: 1.0000000	test: 0.7473436	best: 0.7933884 (64)	total: 13.5s	remaining: 9.22s
595:	learn: 1.0000000	t

682:	learn: 1.0000000	test: 0.7473436	best: 0.7933884 (64)	total: 15.5s	remaining: 7.21s
683:	learn: 1.0000000	test: 0.7473436	best: 0.7933884 (64)	total: 15.6s	remaining: 7.19s
684:	learn: 1.0000000	test: 0.7473436	best: 0.7933884 (64)	total: 15.6s	remaining: 7.16s
685:	learn: 1.0000000	test: 0.7473436	best: 0.7933884 (64)	total: 15.6s	remaining: 7.14s
686:	learn: 1.0000000	test: 0.7485242	best: 0.7933884 (64)	total: 15.6s	remaining: 7.12s
687:	learn: 1.0000000	test: 0.7485242	best: 0.7933884 (64)	total: 15.6s	remaining: 7.09s
688:	learn: 1.0000000	test: 0.7473436	best: 0.7933884 (64)	total: 15.7s	remaining: 7.07s
689:	learn: 1.0000000	test: 0.7485242	best: 0.7933884 (64)	total: 15.7s	remaining: 7.05s
690:	learn: 1.0000000	test: 0.7473436	best: 0.7933884 (64)	total: 15.7s	remaining: 7.03s
691:	learn: 1.0000000	test: 0.7473436	best: 0.7933884 (64)	total: 15.7s	remaining: 7s
692:	learn: 1.0000000	test: 0.7473436	best: 0.7933884 (64)	total: 15.8s	remaining: 6.98s
693:	learn: 1.0000000	te

778:	learn: 1.0000000	test: 0.7461629	best: 0.7933884 (64)	total: 17.7s	remaining: 5.03s
779:	learn: 1.0000000	test: 0.7461629	best: 0.7933884 (64)	total: 17.8s	remaining: 5.01s
780:	learn: 1.0000000	test: 0.7461629	best: 0.7933884 (64)	total: 17.8s	remaining: 4.99s
781:	learn: 1.0000000	test: 0.7461629	best: 0.7933884 (64)	total: 17.8s	remaining: 4.96s
782:	learn: 1.0000000	test: 0.7461629	best: 0.7933884 (64)	total: 17.8s	remaining: 4.94s
783:	learn: 1.0000000	test: 0.7449823	best: 0.7933884 (64)	total: 17.8s	remaining: 4.92s
784:	learn: 1.0000000	test: 0.7461629	best: 0.7933884 (64)	total: 17.9s	remaining: 4.89s
785:	learn: 1.0000000	test: 0.7461629	best: 0.7933884 (64)	total: 17.9s	remaining: 4.87s
786:	learn: 1.0000000	test: 0.7461629	best: 0.7933884 (64)	total: 17.9s	remaining: 4.85s
787:	learn: 1.0000000	test: 0.7449823	best: 0.7933884 (64)	total: 17.9s	remaining: 4.83s
788:	learn: 1.0000000	test: 0.7449823	best: 0.7933884 (64)	total: 18s	remaining: 4.8s
789:	learn: 1.0000000	te

872:	learn: 1.0000000	test: 0.7426210	best: 0.7933884 (64)	total: 19.9s	remaining: 2.89s
873:	learn: 1.0000000	test: 0.7426210	best: 0.7933884 (64)	total: 19.9s	remaining: 2.87s
874:	learn: 1.0000000	test: 0.7414404	best: 0.7933884 (64)	total: 19.9s	remaining: 2.84s
875:	learn: 1.0000000	test: 0.7414404	best: 0.7933884 (64)	total: 19.9s	remaining: 2.82s
876:	learn: 1.0000000	test: 0.7414404	best: 0.7933884 (64)	total: 19.9s	remaining: 2.8s
877:	learn: 1.0000000	test: 0.7414404	best: 0.7933884 (64)	total: 20s	remaining: 2.77s
878:	learn: 1.0000000	test: 0.7414404	best: 0.7933884 (64)	total: 20s	remaining: 2.75s
879:	learn: 1.0000000	test: 0.7414404	best: 0.7933884 (64)	total: 20s	remaining: 2.73s
880:	learn: 1.0000000	test: 0.7414404	best: 0.7933884 (64)	total: 20s	remaining: 2.71s
881:	learn: 1.0000000	test: 0.7414404	best: 0.7933884 (64)	total: 20.1s	remaining: 2.68s
882:	learn: 1.0000000	test: 0.7414404	best: 0.7933884 (64)	total: 20.1s	remaining: 2.66s
883:	learn: 1.0000000	test: 0.

968:	learn: 1.0000000	test: 0.7426210	best: 0.7933884 (64)	total: 22s	remaining: 705ms
969:	learn: 1.0000000	test: 0.7426210	best: 0.7933884 (64)	total: 22.1s	remaining: 682ms
970:	learn: 1.0000000	test: 0.7426210	best: 0.7933884 (64)	total: 22.1s	remaining: 659ms
971:	learn: 1.0000000	test: 0.7426210	best: 0.7933884 (64)	total: 22.1s	remaining: 637ms
972:	learn: 1.0000000	test: 0.7426210	best: 0.7933884 (64)	total: 22.1s	remaining: 614ms
973:	learn: 1.0000000	test: 0.7414404	best: 0.7933884 (64)	total: 22.1s	remaining: 591ms
974:	learn: 1.0000000	test: 0.7414404	best: 0.7933884 (64)	total: 22.2s	remaining: 568ms
975:	learn: 1.0000000	test: 0.7414404	best: 0.7933884 (64)	total: 22.2s	remaining: 546ms
976:	learn: 1.0000000	test: 0.7426210	best: 0.7933884 (64)	total: 22.2s	remaining: 523ms
977:	learn: 1.0000000	test: 0.7426210	best: 0.7933884 (64)	total: 22.2s	remaining: 500ms
978:	learn: 1.0000000	test: 0.7426210	best: 0.7933884 (64)	total: 22.3s	remaining: 478ms
979:	learn: 1.0000000	t